In [1]:
import sys
sys.path.insert(0, '/home/noah/Desktop/large_analysis/ca_analysis')
import numpy as np
import matplotlib.pyplot as plt
import fileread as fr
import predefined as pd
import wave_ops as wo
from scipy.optimize import curve_fit
import time


data=fr.gen_output('/home/noah/Desktop/large_analysis/ca_analysis/cur_data/Run_120-all.dat')[0]
simdata=fr.simulation('/home/noah/Desktop/large_analysis/zeck_simulation/45Caaf.txt')

No data cuts, just looking at number of events for each raw set. For real data this includes quite a bit of noise triggers/pulser events etc. For simulation, this does not sum backscattered events.

In [28]:
table=np.zeros((10,10))
cut=[]

bd,ch=1,3
tot=len(data)
frac= 100 #100.*len(pd.single_pixel(data,bd,ch))/len(data)
cut.append('None')
eastfrac=np.sum(data['board']>2)/len(data)
westfrac=1-eastfrac
table[0,0:2]=tot,frac
print('Fraction of all data on '+pd.pixel(bd,ch)+\
      ' = {:0.2f}'.format(100.*len(pd.single_pixel(data,bd,ch))/len(data)))
#count=0    NOT DOING THIS UNLESS REQUIRED
#while i < len(simdata)-1:
#    j=1
#    backscattering= simdata['entry'][i]==simdata['entry'][i+1]
#    if simdata['pixel'][i]==pix and simdata['detector'][i]==det\
#    and not backscattering:
#        count+=1
#        i+=1
#    while backscattering:
#        if simdata['pixel'][j] == pix and simdata['detector'][j]== det\
#        and 
tot=len(simdata)
frac= 100 #100.*len(pd.single_pixel(data,bd,ch))/len(data)        
table[0,4:6]=tot,frac
print('Fraction of simulation on '+pd.pixel(bd,ch)+\
      ' = {:0.2f}'.format(100*len(pd.sim_single_pixel(simdata,bd,ch))/len(simdata)))

Fraction of all data on 64W = 29.90
Fraction of simulation on 64W = 43.04


Making E>16 keV cut and removing pulser from counts:

In [29]:
cutdata=data[pd.land(data['energy']>100,data['energy']<2000)]
count=len(pd.single_pixel(cutdata,bd,ch))
print('Fraction of all data on '+pd.pixel(bd,ch)+\
      ' = {:0.2f}'.format(100*count/len(cutdata)))
tot=len(cutdata)
frac=100*tot/table[0,0]
table[1,0:2]=tot,frac


simcut=simdata[simdata['energy']>16]
count=len(pd.sim_single_pixel(simcut,bd,ch))
print('Fraction of all data on '+pd.pixel(bd,ch)+\
      ' = {:0.2f}'.format(100*count/len(simcut)))
tot=len(simcut)
frac=tot/table[0,4]*100
table[1,4:6]=tot,frac
cut.append('E>16 keV')

Fraction of all data on 64W = 42.39
Fraction of all data on 64W = 43.20


Making single pixel cut ($\Delta t$ > 1 $\mu $s)

In [30]:
cutdata.sort(order='timestamp')
t0,t1,t2=cutdata['timestamp'][0:-2],cutdata['timestamp'][1:-1],\
        cutdata['timestamp'][2:]
trutharray=pd.land(t2-t1>250,t1-t0>250)
cutdata=cutdata[1:-1][trutharray]
count=len(pd.single_pixel(cutdata,bd,ch))
print('Fraction of all data on '+pd.pixel(bd,ch)+\
      ' = {:0.2f}'.format(100*count/len(cutdata)))
tot=len(cutdata)
frac=tot/table[1,0]*100
table[2,0:2]=tot,frac


simcut=pd.sim_comb_single_pixel(pd.sim_spixel_cut(simdata))
simcut=simcut[simcut['pixel']!=0]
count= len(pd.sim_single_pixel(simcut,bd,ch))
print('Fraction of all data on '+pd.pixel(bd,ch)+\
      ' = {:0.2f}'.format(100*count/len(simcut)))
tot=len(simcut)
frac=tot/table[1,4]*100
table[2,4:6]=tot,frac
cut.append('1 us cut')

Fraction of all data on 64W = 44.11
Fraction of all data on 64W = 42.97


Removing accidental pileup (ONLY APPICABLE TO DATA, simulation unchanged)

In [31]:
cutdata=cutdata[pd.lor(cutdata['pileup']<2,cutdata['pilediff']<60)]
count=len(pd.single_pixel(cutdata,bd,ch))
print('Fraction of all data on '+pd.pixel(bd,ch)+\
      ' = {:0.2f}'.format(100*count/len(cutdata)))

tot=len(cutdata)
frac=tot/table[2,0]*100
table[3,0:2]=tot,frac

count= len(pd.sim_single_pixel(simcut,bd,ch))
print('Fraction of all data on '+pd.pixel(bd,ch)+\
      ' = {:0.2f}'.format(100*count/len(simcut)))
cut.append('Pileup')

Fraction of all data on 64W = 44.60
Fraction of all data on 64W = 42.97


Removing all events with t_0 <900 bins (events with unusual triggers), again not applicable to simulation

In [32]:
cutdata=cutdata[cutdata['t0']>900]
count=len(pd.single_pixel(cutdata,bd,ch))
print('Fraction of all data on '+pd.pixel(bd,ch)+\
      ' = {:0.2f}'.format(100*count/len(cutdata)))
tot=len(cutdata)
frac=tot/table[3,0]*100
table[4,0:2]=tot,frac


count= len(pd.sim_single_pixel(simcut,bd,ch))
print('Fraction of all data on '+pd.pixel(bd,ch)+\
      ' = {:0.2f}'.format(100*count/len(simcut)))
cut.append('t0>900')

Fraction of all data on 64W = 44.72
Fraction of all data on 64W = 42.97


In [38]:
print('Cut','\t\t','Data counts','\t','Data Fraction (%)','\t','Simulation Counts','\t','Simulation Fraction')
print(100*'-')
for i in range(5):
    [s1,s2],[s3,s4]=table[i,0:2],table[i,4:6]
    if s3 == 0:
        s1,s2='{:0.0f}'.format(s1),'{:0.1f}'.format(s2)
        s3,s4='N/A     ','N/A'
    else:
        s1,s2,s3,s4='{:0.0f}'.format(s1),'{:0.1f}'.format(s2),'{:0.0f}'.format(s3),'{:0.2f}'.format(s4)
    if i==1 or i==2:
        spacetab='\t'
    else:
            spacetab='\t\t'
    print (cut[i],spacetab,s1,'\t',s2,'\t\t\t',s3,'\t\t',s4)

Cut 		 Data counts 	 Data Fraction (%) 	 Simulation Counts 	 Simulation Fraction
----------------------------------------------------------------------------------------------------
None 		 4932208 	 100.0 			 8005312 		 100.00
E>16 keV 	 1679611 	 34.1 			 6233546 		 77.87
1 us cut 	 1202737 	 71.6 			 5171304 		 82.96
Pileup 		 1188216 	 98.8 			 N/A      		 N/A
t0>900 		 1184418 	 99.7 			 N/A      		 N/A
